# Практика 6: Атака по переносу (Transfer Attack) на модели ИИ
# Чурсинов Герман ББМО-01-23

## Изучить концепцию атаки по переносу, где противоречивые примеры, созданные для одной модели, используются для атаки на другую модель. Это задание требует создания нескольких моделей, генерации противоречивых примеров для одной модели и проверки их на другой модели.

## Задачи:
1. Загрузить несколько моделей, обученных на датасете MNIST.
2. Изучить теоретические основы атаки по переносу.
3. Реализовать атаку FGSM на одну модель и проверить, как противоречивые примеры влияют на другую модель.
4. Оценить точность обеих моделей на противоречивых примерах и проанализировать переносимость атак.

## Шаги выполнения:
**Шаг 1: Загрузка и создание двух различных моделей**

Создаем две модели на датасете MNIST: одну простую полносвязную сеть и одну свёрточную нейронную сеть (CNN).



In [8]:
# Импорт всех необходимых для работы библиотек
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import numpy as np

In [3]:
# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0
# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Модель 1: Простая полносвязная нейронная сеть
model1 = Sequential([
 Flatten(input_shape=(28, 28)),
 Dense(128, activation='relu'),
 Dense(10, activation='softmax')
 ])
# Компиляция модели
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Обучение модели
model1.fit(train_images, train_labels, epochs=5)
# Сохранение модели
model1.save('mnist_model1.h5')

# Модель 2: Свёрточная нейронная сеть (CNN)
model2 = Sequential([
 Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
 MaxPooling2D((2, 2)),
 Flatten(),
 Dense(128, activation='relu'),
 Dense(10, activation='softmax')
])
# Компиляция модели
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Обучение модели
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)
# Сохранение модели
model2.save('mnist_model2.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8798 - loss: 0.4279
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9630 - loss: 0.1221
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9758 - loss: 0.0794
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9819 - loss: 0.0587
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9856 - loss: 0.0456


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 53s 27ms/step - accuracy: 0.9117 - loss: 0.2972
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 26ms/step - accuracy: 0.9822 - loss: 0.0572
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 78s 24ms/step - accuracy: 0.9904 - loss: 0.0318
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 24ms/step - accuracy: 0.9932 - loss: 0.0211
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - accuracy: 0.9958 - loss: 0.0135


**Шаг 2: Теория атаки по переносу**

Атака по переносу — это метод, при котором противоречивые примеры, созданные для одной модели, применяются к другой модели. Это возможно, потому что модели часто имеют схожие слабости и могут совершать одинаковые ошибки на определенных данных, даже если они обучены на разных архитектурах.

В этом задании мы создадим противоречивые примеры для первой модели с помощью FGSM и затем проверим, насколько эти примеры могут атаковать вторую модель

**Шаг 3: Реализация атаки FGSM на первую модель**

Мы применим атаку FGSM (Fast Gradient Sign Method) к первой модели, чтобы создать противоречивые примеры.

In [16]:
# Функция для реализации FGSM атаки
def fgsm_attack(image, epsilon, gradient):
  # Применение знака градиента к изображению
  perturbed_image = image + epsilon * np.sign(gradient)
  perturbed_image = np.clip(perturbed_image, 0, 1) # Убедиться, что значения остаются в пределах [0, 1]
  return perturbed_image

# Вычисление градиента
def generate_adversarial_example(model, image, label, epsilon):
    # Превращаем изображение в формат, подходящий для модели
    image = tf.convert_to_tensor(image.reshape((1, 28, 28, 1)))
    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label) # ФИКС ОШИБКИ ИЗ ЗАДАНИЯ
    label = tf.convert_to_tensor(label)

    # Вычисление градиента
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)
    gradient = tape.gradient(loss, image)
    # Применяем FGSM
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
    # Убедимся, что adversarial_image имеет правильную форму
    return np.reshape(adversarial_image, (28, 28, 1))

def generate_fgsm_adversarial(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
      image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)))
      if len(labels[i].shape) > 1 and labels[i].shape[1] > 1:
        label = np.argmax(labels[i]) # ФИКС ОШИБКИ ИЗ ЗАДАНИЯ
      label = tf.convert_to_tensor(labels[i])
      # Вычисление градиента
      with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.categorical_crossentropy(labels[i][None], prediction)
      gradient = tape.gradient(loss, image)
      # Применяем FGSM
      adversarial_image = np.reshape(fgsm_attack(image.numpy(), epsilon, gradient.numpy()), (28, 28, 1))
      # Убедимся, что adversarial_image имеет правильную форму

      adversarial_images.append(adversarial_image.reshape(28, 28))
    adversarial_images = np.array(adversarial_images)
    # Проверка формы
    print("Shape of adversarial_images:", adversarial_images.shape)
    return adversarial_images


# Генерация противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_fgsm_adversarial(model1, test_images, test_labels, epsilon)

Shape of adversarial_images: (10000, 28, 28)


**Шаг 4: Оценка противоречивых примеров на обеих моделях**

Теперь мы проверим, как сильно атака PGD влияет на точность модели. Мы создадим набор противоречивых примеров и оценим производительность модели на этих данных.

In [11]:
# Оценка первой модели на противоречивых примерах
test_labels_argmax = np.argmax(test_labels, axis=1) # Преобразование onehot меток в целые числа
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Accuracy of model1 on adversarial examples: {acc1}')

# Оценка второй модели на противоречивых примерах (перенос атаки)
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Accuracy of model2 on adversarial examples from model1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0895 - loss: 6.6602
Accuracy of model1 on adversarial examples: 0.11720000207424164
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9614 - loss: 0.1273
Accuracy of model2 on adversarial examples from model1: 0.9660999774932861


Шаг 5: Анализ переносимости атак

Проанализируйте, насколько успешно атака переносится с одной модели на другую. Вы можете заметить, что несмотря на различия в архитектуре, вторая модель также подвержена атаке, хотя и с другой степенью точности. Дополнительное задание (усложнение)

Попробуйте создать противоречивые примеры для второй модели с помощью FGSM и проверьте их на первой модели.
Оцените, какие модели более уязвимы к атакам по переносу, и предложите возможные решения для усиления защиты.

In [13]:
# Генерация противоречивых примеров для второй модели
adversarial_images_model2 = generate_fgsm_adversarial(model2, test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28, 28), test_labels)
print(f'Accuracy of model1 on adversarial examples from model2: {acc3}')

Shape of adversarial_images: (10000, 28, 28)
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9537 - loss: 0.1492
Accuracy of model1 on adversarial examples from model2: 0.9599000215530396


Результаты:

После выполнения этой практики ожидается, что вторая модель также будет уязвима к противоречивым примерам, созданным для первой модели, но с разной степенью эффективности. Вы также должны увидеть, что атака, созданная для одной модели, может работать на другой модели.
И вот, в ходе выполнения задания мной была реализована атака по переносу, используя противоречивые примеры.

Стоит отметить, что вторая модель оказалась ожидаемо уязвима к противоречивым примерам, которые я создал для первой модели, но оказалась уязвима с разной степенью эффективности. Также можно заметить, что атака, созданная для одной модели, может работать и на другой модели. На основании полученных результатов различные модели могут быть уязвимы к одним и тем же противоречивым примерам, даже если они обучены по-разному.